# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

Peter: which are the first 100 chars?

In [2]:
print(text[:100])


Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hell


## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [3]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.251908396946565
Number of lines: 4258
Average number of words in each line: 11.50164396430249

The sentences 0 to 10:

Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.



## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [4]:

test_text = '''
There’s something magical about Recurrent Neural Networks (RNNs).
I still remember when I trained my first recurrent network for Image Captioning.
Within a few dozen minutes of training my first baby model (with rather arbitrarily-chosen
hyperparameters) started to generate very nice looking descriptions of images that were on
the edge of making sense. Sometimes the ratio of how simple your model is to the quality of
the results you get out of it blows past your expectations, and this was one of those times.
What made this result so shocking at the time was that the common wisdom was that RNNs were
supposed to be difficult to train (with more experience I’ve in fact reached the opposite
conclusion). Fast forward about a year: I’m training RNNs all the time and I’ve witnessed
their power and robustness many times, and yet their magical outputs still find ways of
amusing me. This post is about sharing some of that magic with you.'''

test_text = "||return|| moe_szyslak:  ||left_paren|| into phone ||right_paren||  moe's tavern ||period||  where "

test_text = test_text.lower()
test_text = test_text.split()

In [5]:
from collections import Counter
word_counts = Counter(test_text[:100])
print(word_counts)
sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
print(sorted_vocab)

Counter({'||period||': 1, 'into': 1, '||left_paren||': 1, 'tavern': 1, 'where': 1, "moe's": 1, '||right_paren||': 1, 'moe_szyslak:': 1, '||return||': 1, 'phone': 1})
['||period||', 'into', '||left_paren||', 'tavern', 'where', "moe's", '||right_paren||', 'moe_szyslak:', '||return||', 'phone']


In [6]:
import numpy as np
import problem_unittests as tests

from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # DONE: Implement Function
    print(text[:100])
    word_counts = Counter(text)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = dict(zip(int_to_vocab.values(), int_to_vocab.keys()))
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

['moe_szyslak', "moe's", 'tavern', 'where', 'the', 'elite', 'meet', 'to', 'drink', 'bart_simpson', 'eh', 'yeah', 'hello', 'is', 'mike', 'there', 'last', 'name', 'rotch', 'moe_szyslak', 'hold', 'on', "i'll", 'check', 'mike', 'rotch', 'mike', 'rotch', 'hey', 'has', 'anybody', 'seen', 'mike', 'rotch', 'lately', 'moe_szyslak', 'listen', 'you', 'little', 'puke', 'one', 'of', 'these', 'days', "i'm", 'gonna', 'catch', 'you', 'and', "i'm", 'gonna', 'carve', 'my', 'name', 'on', 'your', 'back', 'with', 'an', 'ice', 'pick', 'moe_szyslak', 'whats', 'the', 'matter', 'homer', "you're", 'not', 'your', 'normal', 'effervescent', 'self', 'homer_simpson', 'i', 'got', 'my', 'problems', 'moe', 'give', 'me', 'another', 'one', 'moe_szyslak', 'homer', 'hey', 'you', 'should', 'not', 'drink', 'to', 'forget', 'your', 'problems', 'barney_gumble', 'yeah', 'you', 'should', 'only', 'drink', 'to']
Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [7]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # DONE: Implement Function
    lookup = {
        '.': '||PERIOD||',
        ',': '||COMMA||',
        '"': '||QUOTATION_MARK||',
        ';': '||SEMICOLON||',
        #':': '||COLON||',
        '!': '||EXCLAMATION_MARK||',
        '?': '||QUESTION_MARK||',
        '(': '||LEFT_PAREN||',
        ')': '||RIGHT_PAREN||',
        '--': '||DASH||',
        '\n': '||RETURN||'
    }
    return lookup

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [8]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

 ||return|| moe_szyslak:  ||left_paren|| into phone ||right_paren||  moe's tavern ||period||  where 
['||return||', 'moe_szyslak:', '||left_paren||', 'into', 'phone', '||right_paren||', "moe's", 'tavern', '||period||', 'where', 'the', 'elite', 'meet', 'to', 'drink', '||period||', '||return||', 'bart_simpson:', 'eh', '||comma||', 'yeah', '||comma||', 'hello', '||comma||', 'is', 'mike', 'there', '||question_mark||', 'last', 'name', '||comma||', 'rotch', '||period||', '||return||', 'moe_szyslak:', '||left_paren||', 'into', 'phone', '||right_paren||', 'hold', 'on', '||comma||', "i'll", 'check', '||period||', '||left_paren||', 'to', 'barflies', '||right_paren||', 'mike', 'rotch', '||period||', 'mike', 'rotch', '||period||', 'hey', '||comma||', 'has', 'anybody', 'seen', 'mike', 'rotch', '||comma||', 'lately', '||question_mark||', '||return||', 'moe_szyslak:', '||left_paren||', 'into', 'phone', '||right_paren||', 'listen', 'you', 'little', 'puke', '||period||', 'one', 'of', 'these', 'days', "

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [9]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [10]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.1
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [20]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # DONE: Implement Function
    
    # Declare placeholders we'll feed into the graph
    input = tf.placeholder(tf.int32, [None, None], name='input')
    target = tf.placeholder(tf.int32, [None, None], name='target')
    
    # Learning_rate
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    
    return input, target, learning_rate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [12]:
# Stack one or more BasicLSTMCells in a MultiRNNCell.
num_layers = 2

# Keep Prob parameter for the LSTM Cells
keep_prob = 1.0

def get_init_cell(batch_size, rnn_size, num_layers = 2, keep_prob = 1.0):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # DONE: Implement Function
    def build_cell(lstm_size, keep_prob):
        # Use a basic LSTM cell
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        
        # Add dropout to the cell if needed
        if keep_prob < 1.0:
            lstm = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        
        return lstm
    
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    
    # Apply initial state
    initial_state = tf.identity(cell.zero_state(batch_size, tf.float32), name='initial_state')
    
    return cell, initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

Before apply we have to build a embedding matrix for the words in the text.
It is needed to apply the 

In [13]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # DONE: Implement Function
    with tf.device('/cpu:0'):
        W = tf.Variable(tf.random_uniform([vocab_size, embed_dim], -1.0, 1.0))
        return tf.nn.embedding_lookup(W, input_data)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [19]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # DONE: Implement Function

    # Run each sequence step through the RNN and collect the outputs
    outputs, state = tf.nn.dynamic_rnn(cell, inputs, initial_state=???)
    state = tf.identity(state, name='final_state')
    
    return outputs, state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [ ]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
        
    # Build the input placeholder tensors
    input, target, learning_rate = get_inputs()
    
    # apply embeding
    input = get_embed(input_data, vocab_size, embed_dim)
    
    # Build the LSTM cell
    num_layers = 2
    keep_prob = 1.0
    cell, initial_state = build_lstm(rnn_size, num_layers, batch_size, keep_prob)

    ### Run the data through the RNN layers
    # First, one-hot encode the input tokens
    x_one_hot = tf.one_hot(self.inputs, num_classes)

    # Run each sequence step through the RNN and collect the outputs
    outputs, final_state = build_rnn(cell, inputs)

    # Get softmax predictions and logits
    
    self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
    
    return None, None


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [65]:
a = np.array(range(1, 61))

batch_size = 4
seq_length = 3

# Get the number of words per step and number of steps we can make
words_per_batch = batch_size * seq_length
num_batches = len(a)//words_per_batch

# Keep only enough words to make full batches
a = a[:num_batches * words_per_batch]

print("a", a)
print('batch_size',batch_size)
print('seq_length',seq_length)

b = list()

for batch_index in range(num_batches):
    batch_start = batch_index * seq_length
    for input_or_output in range(len(['input', 'output'])):
        input_or_output_start = input_or_output
        for seq_batch_index in range(batch_size):
            seq_batch_start = seq_batch_index * words_per_batch
            for seq_index in range(seq_length):
                b.append((batch_start + input_or_output_start + seq_batch_start + seq_index))

                
print("b", b)

b = np.array(b)

b = b.reshape(-1, 2, batch_size, seq_length)
print('b.shape', b.shape)
print("b", b)


#a = np.array([1, 2, 7, 8, 13, 14, 2, 3, 8, 9, 14, 15, 13, 14, 15, 3, 4, 5, 6, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40])


a [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50
 51 52 53 54 55 56 57 58 59 60]
batch_size 4
seq_length 3
b [0, 1, 2, 12, 13, 14, 24, 25, 26, 36, 37, 38, 1, 2, 3, 13, 14, 15, 25, 26, 27, 37, 38, 39, 3, 4, 5, 15, 16, 17, 27, 28, 29, 39, 40, 41, 4, 5, 6, 16, 17, 18, 28, 29, 30, 40, 41, 42, 6, 7, 8, 18, 19, 20, 30, 31, 32, 42, 43, 44, 7, 8, 9, 19, 20, 21, 31, 32, 33, 43, 44, 45, 9, 10, 11, 21, 22, 23, 33, 34, 35, 45, 46, 47, 10, 11, 12, 22, 23, 24, 34, 35, 36, 46, 47, 48, 12, 13, 14, 24, 25, 26, 36, 37, 38, 48, 49, 50, 13, 14, 15, 25, 26, 27, 37, 38, 39, 49, 50, 51]
b.shape (5, 2, 4, 3)
b [[[[ 0  1  2]
   [12 13 14]
   [24 25 26]
   [36 37 38]]

  [[ 1  2  3]
   [13 14 15]
   [25 26 27]
   [37 38 39]]]


 [[[ 3  4  5]
   [15 16 17]
   [27 28 29]
   [39 40 41]]

  [[ 4  5  6]
   [16 17 18]
   [28 29 30]
   [40 41 42]]]


 [[[ 6  7  8]
   [18 19 20]
   [30 31 32]
   [42 43 44]]

  [[ 7 

In [17]:
import problem_unittests as tests
import numpy as np
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # DONE: Implement Function
    
    # Transform input int_text into an np.array()
    a = np.array(int_text)
    
    # Get the number of words per batch and number of batches we can make
    words_per_batch = batch_size * seq_length
    num_batches = len(a)//words_per_batch

    # Keep only enough words to make full batches
    a = a[:num_batches * words_per_batch]

    
    b = []
    for batch_index in range(num_batches):
        batch_start = batch_index * seq_length
        for input_or_output in range(len(['input', 'output'])):
            input_or_output_start = input_or_output 
            for batch_channel_index in range(batch_size):
                batch_channel_start = batch_start + batch_channel_index * num_batches * seq_length
                for seq_index in range(seq_length):
                    b.append(a[(batch_channel_start + seq_index  + input_or_output_start) % len(a)])


    #print("num_batches", num_batches)
    #print("batch_size", batch_size)
    #print("seq_length", seq_length)
    

    b = np.array(b)
    #print("b", b[:30])

    #b = np.array(b)
    b = b.reshape(-1, 2, batch_size, seq_length)
    #b = b.reshape(-1, batch_size, seq_length)
    #print("b", b)
    return b


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [ ]:
# Number of Epochs
num_epochs = 50
# Batch Size
batch_size = 50
# RNN Size
rnn_size = 128
# Embedding Dimension Size
embed_dim = 64
# Sequence Length
seq_length = 50
# Learning Rate
learning_rate = 2e-3
# Show stats for every n number of batches
show_every_n_batches = 1

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forms](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [ ]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    return None, None, None, None


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [ ]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # DONE: Implement Function
    index = np.random.choice(len(int_to_vocab), 1, p=probabilities)[0]
    return int_to_vocab[index]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [ ]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.